In [160]:
import pandas as pd
import numpy as np 
from dbfread import DBF
import psycopg2

# Colunas de dados necessario para criar o banco de dados INNOVARO

## colunas:

1- Código <br>
2- Nome <br>
3- classe <br>
4- Classificação fiscal <br>
5- UN <br>
6- Procedência <br>
7- Fabricante <br>
8- Referência fabricante <br>
9- Marca <br>
10- Modelo <br>
11- Coleção/Família <br>
12- Cor <br>
13- Tamanho <br>
14- Quantidade P/Volume <br>
15- Quantidade P/Vol Compra <br>
16- Data em que o produto saiu de linha <br>
17- Informador <br>
18- Data em que o produto tornou-se inativo <br>
19- inativador <br>
20- Material <br>
21- Referência principal <br>
22- Descriçao Genérica <br>
23- Cest <br>
24- Tipo de Medicamento <br>
25- Outras Caracteristicas <br>
26- Validade(meses)

## upload SDB Principal 

In [4]:
estoq001 = DBF('ESTOQ001.dbf', encoding='latin1')
estoq001 = pd.DataFrame(estoq001)

## visualização do banco de dados

In [6]:
estoq001.head(3)

,TQCLASSI,TQCODIGO,TQGRADE,TQDESCRI,TQCODBAR,TQEMBENT,TQENTQTD,TQEMBSAI,TQSAIQTD,TQQTDMAX,...,TQIDPROM,TQQTDPRO,TQVALPRO,TQRASTRO,TQOPERAD,TQULTPRE,TQMARCA,TQBENEFI,TQDESONE,TQALIFCP
0,10.033.052,1,UN,TORQUES ARMADOR,,UN,1,UN,1,0,...,0,0.0,0.0,,37,11.16,,,,0.0
1,10.019.022,2,UN,ACESSORIO VERAO,,UN,1,UN,1,0,...,0,0.0,0.0,,37,5.16,,,,0.0
2,10.033.052,3,UN,X 8EQUOT,,UN,1,UN,1,0,...,0,0.0,0.0,,37,0.00,,,,0.0


## identificação de informações das colunas do dataframe

In [7]:
estoq001.info(verbose = True, show_counts= True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86475 entries, 0 to 86474
Data columns (total 102 columns):
 #    Column     Non-Null Count  Dtype  
---   ------     --------------  -----  
 0    TQCLASSI   86475 non-null  object 
 1    TQCODIGO   86475 non-null  object 
 2    TQGRADE    86475 non-null  object 
 3    TQDESCRI   86475 non-null  object 
 4    TQCODBAR   86475 non-null  object 
 5    TQEMBENT   86475 non-null  object 
 6    TQENTQTD   86475 non-null  int64  
 7    TQEMBSAI   86475 non-null  object 
 8    TQSAIQTD   86475 non-null  int64  
 9    TQQTDMAX   86475 non-null  int64  
 10   TQQTDMIN   86475 non-null  int64  
 11   TQSALDO    85689 non-null  float64
 12   TQULTENT   8417 non-null   object 
 13   TQULTSAI   27694 non-null  object 
 14   TQPREVIS   0 non-null      object 
 15   TQPRECUS   86475 non-null  float64
 16   TQPREVEN   86475 non-null  float64
 17   TQSUBSTI   86475 non-null  float64
 18   TQTRIBUT   86475 non-null  object 
 19   TQALIQUO   86475 non-nu

## seleção de colunas para o banco de dados do INNOVARO 

deverá ser inserido apenas as colunas que queremos visualizar no banco de dados do Prosuper

In [9]:
colunas = ['TQCODIGO', 'TQDESCRI', 'TQGRADE', 'TQOBSERV', 'TQPRECUS', 'TQPREVEN', 'TQNCM', 'TQCFOP', 'TQCEST', 'TQGRUPO', 'TQSGRUPO']

## filtrando as colunas no banco de dados

In [53]:
estoq001_filtro = estoq001[colunas]

In [54]:
estoq001_filtro

,TQCODIGO,TQDESCRI,TQGRADE,TQOBSERV,TQPRECUS,TQPREVEN,TQNCM,TQCFOP,TQCEST,TQGRUPO,TQSGRUPO
0,1,TORQUES ARMADOR,UN,TORQUES ARMADOR,0.000,9.99,8203.20.10,5405,0800800,LEDS,LEDS/LED
1,2,ACESSORIO VERAO,UN,ACESSORIO VERAO,0.000,0.00,8205.40.00,5102,,BIJUTERIAS,BIJUTERIA/ACESSORIOS VERAO
2,3,X 8EQUOT,UN,CHAVE DE FENDA 1/4EQUOT,0.000,5.80,8205.40.00,5405,,LEDS,LEDS/LED
3,4,CHAVE PHILLIPS 1/4,UN,CHAVE PHILLIPS 1/4EQUOT,0.000,5.16,8205.40.00,5405,,LEDS,LEDS/FERRAMENTA
4,5,X 8EQUOT,UN,CHAVE PHILLIPS 1/4EQUOT,0.000,5.80,8205.40.00,5405,,LEDS,LEDS/FERRAMENTA
...,...,...,...,...,...,...,...,...,...,...,...
86470,91691,TELA PARA PINTURA,UN,TELA PARA PINTURA 20X20,1.136,0.00,4414.10.00,5102,,PAPELARIA,PAPELARIA/PAPELARIA
86471,91692,TELA PARA PINTURA,UN,TELA PARA PINTURA 24X30,1.563,0.00,4414.10.00,5102,,PAPELARIA,PAPELARIA/PAPELARIA
86472,91693,TELA PARA PINTURA,UN,TELA PARA PINTURA,2.160,0.00,4414.10.00,5102,,PAPELARIA,PAPELARIA/PAPELARIA
86473,91694,KINDER OVO JOY,UN,CHOC KINDER OVO JOY,0.000,0.00,1806.90.00,5102,1700400,CONVENIENCIA,CONVENIENCIA/ALIMENTOS


## identificando valores do dataframe estoq001_filtro

In [55]:
estoq001_filtro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86475 entries, 0 to 86474
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   TQCODIGO  86475 non-null  object 
 1   TQDESCRI  86475 non-null  object 
 2   TQGRADE   86475 non-null  object 
 3   TQOBSERV  86475 non-null  object 
 4   TQPRECUS  86475 non-null  float64
 5   TQPREVEN  86475 non-null  float64
 6   TQNCM     86475 non-null  object 
 7   TQCFOP    86475 non-null  object 
 8   TQCEST    86475 non-null  object 
 9   TQGRUPO   86475 non-null  object 
 10  TQSGRUPO  86475 non-null  object 
dtypes: float64(2), object(9)
memory usage: 7.3+ MB


## renomeando colunas do dataframe

In [56]:
estoq001_filtro.rename({'TQCODIGO':'CODIGO', 'TQDESCRI':'DESCRICAO', 'TQGRADE' : 'GRADE', 'TQOBSERV':'OBSERVAÇÃO/DESCRIÇÃO COMPLETA', 'TQPRECUS':'CUSTO', 'TQPREVEN':'VENDA', 'TQNCM':'NCM', 'TQCFOP': 'CFOP',
                           'TQCEST':'CEST', 'TQGRUPO':'SETOR', 'TQSGRUPO': 'SUB-GRUPO'}, axis = 1 , inplace = True)

C:\Users\murillo\AppData\Local\Temp\ipykernel_5924\194538697.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  estoq001_filtro.rename({'TQCODIGO':'CODIGO', 'TQDESCRI':'DESCRICAO', 'TQGRADE' : 'GRADE', 'TQOBSERV':'OBSERVAÇÃO/DESCRIÇÃO COMPLETA', 'TQPRECUS':'CUSTO', 'TQPREVEN':'VENDA', 'TQNCM':'NCM', 'TQCFOP': 'CFOP',


## limpando e convertendo dados do dataframe

In [22]:
estoq001_filtro['CODIGO'] = estoq001_filtro['CODIGO'].apply(lambda x: x.replace('.', ''))

C:\Users\murillo\AppData\Local\Temp\ipykernel_5924\213038307.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  estoq001_filtro['CODIGO'] = estoq001_filtro['CODIGO'].apply(lambda x: x.replace('.', ''))


In [48]:
estoq001_filtro['CODIGO'] = estoq001_filtro['CODIGO'].apply(str)

C:\Users\murillo\AppData\Local\Temp\ipykernel_5924\3954982370.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  estoq001_filtro['CODIGO'] = estoq001_filtro['CODIGO'].apply(str)


## identificação dos tipos após conversão

In [104]:
estoq001_filtro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86475 entries, 0 to 86474
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   CODIGO                         86475 non-null  object 
 1   DESCRICAO                      86475 non-null  object 
 2   GRADE                          86475 non-null  object 
 3   OBSERVAÇÃO/DESCRIÇÃO COMPLETA  86475 non-null  object 
 4   CUSTO                          86475 non-null  float64
 5   VENDA                          86475 non-null  float64
 6   NCM                            86475 non-null  object 
 7   CFOP                           86475 non-null  object 
 8   CEST                           86475 non-null  object 
 9   SETOR                          86475 non-null  object 
 10  SUB-GRUPO                      86475 non-null  object 
dtypes: float64(2), object(9)
memory usage: 7.3+ MB


## DataFrame ESTOQ001 CORRIGIDO.

# UPLOAD BANCO DE DADOS COM CODIGOS DE BARRAS

In [58]:
escodbar = DBF('ESCODBAR.dbf', encoding= 'latin1')
escodbar = pd.DataFrame(escodbar)

## visualizando o dataframe de codigos de barras 

In [59]:
escodbar.head(3)

,CBCODBAR,CBCLASSI,CBCODIGO,CBGRADE,CBDESCRI,CBNOMANT,CBID,CBDUPLIC,CBVASILH
0,93,10.033.052,35252,UN,92 PAR20 7W B/Q,LAMPADA PAR 20 7W 3500K,1,P,
1,355,10.020.022,11463,UN,VIOLINHA COLORIDA DE,VIOLINHA COLORIDA DE PLASTICO,2,P,
2,356,10.020.022,11464,UN,GUITARRINHA COLORIDA,GUITARRINHA COLORIDA DE PLASTICO,3,P,


# alterando o nome das colunas do dataframe

In [60]:
escodbar.rename({'CBCODBAR': 'EAN', 'CBCLASSI':'CLASSIFICAÇÃO', 'CBCODIGO': 'CODIGO', 'CBGRADE': 'GRADE', 'CBDESCRI':'DESCRIÇÃO', 'CBNOMANT':'OBSERVAÇÃO/DESCRIÇÃO RESUMIDA',
                    'CBID':'ID' , 'CBDUPLIC': 'DUPLICATAS', 'CBVASILH': 'VASILHAME'}, axis = 1, inplace= True)

## VISUALIZANDO DADOS DO DATAFRAME

In [103]:
escodbar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143045 entries, 0 to 143044
Data columns (total 9 columns):
 #   Column                         Non-Null Count   Dtype 
---  ------                         --------------   ----- 
 0   EAN                            143045 non-null  object
 1   CLASSIFICAÇÃO                  143045 non-null  object
 2   CODIGO                         143045 non-null  object
 3   GRADE                          143045 non-null  object
 4   DESCRIÇÃO                      143045 non-null  object
 5   OBSERVAÇÃO/DESCRIÇÃO RESUMIDA  143045 non-null  object
 6   ID                             143045 non-null  int64 
 7   DUPLICATAS                     143045 non-null  object
 8   VASILHAME                      143045 non-null  object
dtypes: int64(1), object(8)
memory usage: 9.8+ MB


# convertendo dados do dataframe para os tipos corretos

In [43]:
escodbar['CODIGO'] = escodbar['CODIGO'].apply(lambda x: x.replace('.', ''))

In [44]:
escodbar['CODIGO'] = escodbar['CODIGO'].apply(np.int64)

ValueError: invalid literal for int() with base 10: ''

In [105]:
bancoFinal = pd.merge(estoq001_filtro, escodbar , on = 'CODIGO', how= 'inner')

# visualizando o banco de dados final

In [106]:
bancoFinal.head(25)

,CODIGO,DESCRICAO,GRADE_x,OBSERVAÇÃO/DESCRIÇÃO COMPLETA,CUSTO,VENDA,NCM,CFOP,CEST,SETOR,SUB-GRUPO,EAN,CLASSIFICAÇÃO,GRADE_y,DESCRIÇÃO,OBSERVAÇÃO/DESCRIÇÃO RESUMIDA,ID,DUPLICATAS,VASILHAME
0,1,TORQUES ARMADOR,UN,TORQUES ARMADOR,0.0,9.99,8203.20.10,5405,0800800,LEDS,LEDS/LED,7899842205871,10.033.052,UN,TORQUES ARMADOR,TORQUES ARMADOR,143505,R,
1,1,TORQUES ARMADOR,UN,TORQUES ARMADOR,0.0,9.99,8203.20.10,5405,0800800,LEDS,LEDS/LED,179874,10.033.052,UN,TORQUES ARMADOR,TORQUES ARMADOR,143530,R,
2,2,ACESSORIO VERAO,UN,ACESSORIO VERAO,0.0,0.00,8205.40.00,5102,,BIJUTERIAS,BIJUTERIA/ACESSORIOS VERAO,34454,10.019.022,UN,ACESSORIO VERAO,ACESSORIO VERAO,102751,R,
3,3,X 8EQUOT,UN,CHAVE DE FENDA 1/4EQUOT,0.0,5.80,8205.40.00,5405,,LEDS,LEDS/LED,7897373164162,10.033.052,UN,X 8EQUOT,CHAVE DE FENDA 1/4EQUOT,31437,T,
4,3,X 8EQUOT,UN,CHAVE DE FENDA 1/4EQUOT,0.0,5.80,8205.40.00,5405,,LEDS,LEDS/LED,34455,10.033.052,UN,X 8EQUOT,CHAVE DE FENDA 1/4EQUOT,102752,P,
5,4,CHAVE PHILLIPS 1/4,UN,CHAVE PHILLIPS 1/4EQUOT,0.0,5.16,8205.40.00,5405,,LEDS,LEDS/FERRAMENTA,7897373164100,10.033.050,UN,X 5EQUOT,CHAVE PHILLIPS 1/4EQUOT,31434,T,
6,4,CHAVE PHILLIPS 1/4,UN,CHAVE PHILLIPS 1/4EQUOT,0.0,5.16,8205.40.00,5405,,LEDS,LEDS/FERRAMENTA,34457,10.001.002,UN,CHAVE PHILLIPS 1/4,CHAVE PHILLIPS 1/4EQUOT,102754,P,
7,5,X 8EQUOT,UN,CHAVE PHILLIPS 1/4EQUOT,0.0,5.80,8205.40.00,5405,,LEDS,LEDS/FERRAMENTA,7897373164124,10.033.050,UN,X 8EQUOT,CHAVE PHILLIPS 1/4EQUOT,31435,T,
8,5,X 8EQUOT,UN,CHAVE PHILLIPS 1/4EQUOT,0.0,5.80,8205.40.00,5405,,LEDS,LEDS/FERRAMENTA,7890000012430,10.033.050,UN,X 8EQUOT,CHAVE PHILLIPS 1/4EQUOT,139840,T,
9,5,X 8EQUOT,UN,CHAVE PHILLIPS 1/4EQUOT,0.0,5.80,8205.40.00,5405,,LEDS,LEDS/FERRAMENTA,374316,10.033.050,UN,X 8EQUOT,CHAVE PHILLIPS 1/4EQUOT,143532,T,


In [107]:
query= bancoFinal

In [108]:
query.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 168827 entries, 0 to 168826
Data columns (total 19 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CODIGO                         168827 non-null  object 
 1   DESCRICAO                      168827 non-null  object 
 2   GRADE_x                        168827 non-null  object 
 3   OBSERVAÇÃO/DESCRIÇÃO COMPLETA  168827 non-null  object 
 4   CUSTO                          168827 non-null  float64
 5   VENDA                          168827 non-null  float64
 6   NCM                            168827 non-null  object 
 7   CFOP                           168827 non-null  object 
 8   CEST                           168827 non-null  object 
 9   SETOR                          168827 non-null  object 
 10  SUB-GRUPO                      168827 non-null  object 
 11  EAN                            168827 non-null  object 
 12  CLASSIFICAÇÃO                 

In [116]:
query['VENDA'] = query['VENDA'].apply(str) 

In [118]:
selection = query['VENDA'].str.contains('.49', regex=True)
valores49 = query[selection]

In [65]:
bancoFinal.shape

(168827, 19)

In [122]:
valores49['VENDA'] = valores49['VENDA'].apply(float) 

C:\Users\murillo\AppData\Local\Temp\ipykernel_5924\3677157388.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valores49['VENDA'] = valores49['VENDA'].apply(float)


In [124]:
valores49.head()

,CODIGO,DESCRICAO,GRADE_x,OBSERVAÇÃO/DESCRIÇÃO COMPLETA,CUSTO,VENDA,NCM,CFOP,CEST,SETOR,SUB-GRUPO,EAN,CLASSIFICAÇÃO,GRADE_y,DESCRIÇÃO,OBSERVAÇÃO/DESCRIÇÃO RESUMIDA,ID,DUPLICATAS,VASILHAME
20,10,PARATY FACA COZINHA,UN,PARATY FACA COZINHA,0.00,8.49,8211.91.00,5405,,UTENSILIOS,UTENSILIOS/COZINHA,7897240501007,10.049.062,UN,PARATY FACA COZINHA,PARATY FACA COZINHA,30229,R,
21,10,PARATY FACA COZINHA,UN,PARATY FACA COZINHA,0.00,8.49,8211.91.00,5405,,UTENSILIOS,UTENSILIOS/COZINHA,27038,10.049.062,UN,PARATY FACA COZINHA,PARATY FACA COZINHA,91930,T,
87,45,JOGO DE LUVA CASU,UN,JOGO DE LUVA CASUAL,9.14,17.49,3506.99.00,5102,,BIJUTERIAS,BIJUTERIAS/ACESS FRIO,7899259698259,10.019.014,UN,JOGO DE LUVA CASU,JOGO DE LUVA CASUAL,53367,T,
88,45,JOGO DE LUVA CASU,UN,JOGO DE LUVA CASUAL,9.14,17.49,3506.99.00,5102,,BIJUTERIAS,BIJUTERIAS/ACESS FRIO,12884,10.019.014,UN,JOGO DE LUVA CASU,JOGO DE LUVA CASUAL,86074,P,
99,51,KIT MEIAS,UN,KIT MEIAS SORTIDAS,6.57,11.49,6115.10.22,5102,,BIJUTERIAS,BIJUTERIAS/ACESS FRIO,7891290544823,10.019.014,UN,KIT MEIAS,KIT MEIAS SORTIDAS,13084,P,


In [129]:
selection = valores49['VENDA'] > 10.00
valores49 = valores49[selection]

In [131]:
valores49

,CODIGO,DESCRICAO,GRADE_x,OBSERVAÇÃO/DESCRIÇÃO COMPLETA,CUSTO,VENDA,NCM,CFOP,CEST,SETOR,SUB-GRUPO,EAN,CLASSIFICAÇÃO,GRADE_y,DESCRIÇÃO,OBSERVAÇÃO/DESCRIÇÃO RESUMIDA,ID,DUPLICATAS,VASILHAME
87,45,JOGO DE LUVA CASU,UN,JOGO DE LUVA CASUAL,9.14,17.49,3506.99.00,5102,,BIJUTERIAS,BIJUTERIAS/ACESS FRIO,7899259698259,10.019.014,UN,JOGO DE LUVA CASU,JOGO DE LUVA CASUAL,53367,T,
88,45,JOGO DE LUVA CASU,UN,JOGO DE LUVA CASUAL,9.14,17.49,3506.99.00,5102,,BIJUTERIAS,BIJUTERIAS/ACESS FRIO,12884,10.019.014,UN,JOGO DE LUVA CASU,JOGO DE LUVA CASUAL,86074,P,
99,51,KIT MEIAS,UN,KIT MEIAS SORTIDAS,6.57,11.49,6115.10.22,5102,,BIJUTERIAS,BIJUTERIAS/ACESS FRIO,7891290544823,10.019.014,UN,KIT MEIAS,KIT MEIAS SORTIDAS,13084,P,
100,51,KIT MEIAS,UN,KIT MEIAS SORTIDAS,6.57,11.49,6115.10.22,5102,,BIJUTERIAS,BIJUTERIAS/ACESS FRIO,14630,10.019.014,UN,KIT MEIAS,KIT MEIAS SORTIDAS,86777,P,
158,81,SHAKEIRA CANDY,UN,SHAKEIRA CANDY CRUSH PLASUTIL,6.13,13.49,3924.10.00,5102,1400600,BIJUTERIAS,BIJUTERIAS/ACESS FRIO,7896042064239,10.019.014,UN,SHAKEIRA CANDY,SHAKEIRA CANDY CRUSH PLASUTIL,21019,T,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167674,90938,APONTADOR CROC CROC,UN,APONTADOR CROC CROC TWIST 2 FUROS DISPLAY,6.30,11.49,8214.10.00,5102,2005300,PAPELARIA,PAPELARIA/PAPELARIA,3154140184117,10.040.057,UN,APONTADOR CROC CROC,APONTADOR CROC CROC TWIST 2 FUROS DISPLA,153992,R,
167715,90979,LIXEIRA C/ PEDAL INO,UN,LIXEIRA C/ PEDAL INOX 12 LITROS - CM4,79.44,149.99,7323.93.00,5102,1005900,UTILIDADES,UTILIDADES/UTILIDADES,7908486103594,10.050.068,UN,LIXEIRA C/ PEDAL INO,LIXEIRA C/ PEDAL INOX 12 LITROS - CM4,154033,R,
167911,91132,MOCHILA C/ CARRINHO,UN,MOCHILA COM CARRINHO LM2050K,46.98,149.99,4202.92.00,6102,1900400,PAPELARIA,PAPELARIA/PAPELARIA,7899753539119,10.040.057,UN,MOCHILA C/ CARRINHO,MOCHILA COM CARRINHO LM2050K,154230,R,
168174,91238,BASE LIQUIDA LUMINO\r,UN,BASE LIQUIDA LUMINOUS FENZZA MAKE\r,8.71,17.49,3304.99.90,5102,2001500,BIJUTERIAS,BIJUTERIAS/MAQUIAGEM,7899360260437,10.019.020,UN,BASE LIQUIDA LUMINO\r,BASE LIQUIDA LUMINOUS FENZZA MAKE\r,154401,R,


In [133]:
valores49.to_excel('produtos com valores de venda ,49.xlsx', sheet_name = 'sheet1', index = False)

In [137]:
query['VENDA'] = query['VENDA'].apply(float) 

In [138]:
query.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 168827 entries, 0 to 168826
Data columns (total 19 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CODIGO                         168827 non-null  object 
 1   DESCRICAO                      168827 non-null  object 
 2   GRADE_x                        168827 non-null  object 
 3   OBSERVAÇÃO/DESCRIÇÃO COMPLETA  168827 non-null  object 
 4   CUSTO                          168827 non-null  float64
 5   VENDA                          168827 non-null  float64
 6   NCM                            168827 non-null  object 
 7   CFOP                           168827 non-null  object 
 8   CEST                           168827 non-null  object 
 9   SETOR                          168827 non-null  object 
 10  SUB-GRUPO                      168827 non-null  object 
 11  EAN                            168827 non-null  object 
 12  CLASSIFICAÇÃO                 

In [151]:
valores = [
7898684811301,
7898505948414,
7897064411841,
7897064411834,
7898684811332,
7908148590373,
7891100045069,
7898008425986,
277761,
7898008427843,
7899527153060,
7908148590359,
281423,
7898545625528,
7908148511644,
7899790933666,
6200000151793,
7896264701035,
7896264770109,
7908076229208,
7908076229185,
230391,
7898543246107,
230421,
230223,
230230,
230353,
7898545324421,
230346,
7898545324964,
230254,
7896264701059,
7896264761008,
7898008427911,
7898643583553,
7898643581207,
7898643581382,
7898209567782,
7898209567751,
7898643585472,
7898396000604,
7898541953205,
7898543141105,
7898542019016,
7898543141112,
7898543141075,
7898545100032,
7898544146307,
7898545690045,
7898542019078,
7898541212067,
7898541114002,
7898545100018,
7898545100070,
7898545100025,
7899670808169,
7898542019092,
7898542019030,
7898542019054,
7898541114026,
7898542019061,
7898542019047,
7899527109197,
7896554413587,
7896554413587,
7896554411279,
7896554411606,
7896554413587,
7896554411255,
7890002025322,
7890002025308,
7898953747188,
7896020630050,
7898953747164,
32081,
7898953747089,
7897064410165,
7898008425986,
7899527153060,
6200000151793,
7908399124112,
7898505948414,
7898505948421,
7908148511644,
7908148590373,
7908148590366,
7908148590359,
7891100045069,
7899670810131,
7898008427843,
7896484180139,
7899790933659,
7899790933666,
7898684811301,
7897064411841,
7898545625528,
]

In [141]:
query['EAN'] = query['EAN'].apply(np.int64) 

In [152]:
query['filtro'] = query['EAN'].apply(lambda x: True if x in valores else False)

In [153]:
list(query['filtro'].drop_duplicates())

[False, True]

In [154]:
selection = query['filtro'] == True
valores_finais = query[selection]

In [155]:
colunas = ['CODIGO', 'EAN', 'DESCRICAO', 'CUSTO', 'VENDA']

In [156]:
valores_finais = valores_finais[colunas]

In [157]:
valores_finais.to_excel('queima de estoque.xlsx', index= False)

In [158]:
sql = pd.read_json('maindata_nfedatainformations.json')

C:\Users\murillo\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\dtypes\astype.py:189: RuntimeWarning: invalid value encountered in cast
  return values.astype(dtype, copy=copy)


In [161]:
sql.head(3)

,id,reference,ean,eantributed,description,amount,realcost,costcalculate,unitvalue,totalvalue,...,coffinsvalue,aliqfrete,fretevalue,nfenum,fornecedor,cnpjforn,lojaentrada,cnpjlojaentrada,keynfe,frete_values
0,300200001417896058500711AMENDOIM 70G CONF COLO...,3002000014,17896058500711,7896058500714,AMENDOIM 70G CONF COLORIDO DORI (CX 30UN)(FD10...,2.0,56.99,56.99,56.99,113.98,...,7.10,0,0,267167,JJGG-DISTRIBUIDORA ASSIS ALIMENTOS LTDA,9553327000165,CASA DA MAMAE LOJA DE DEP VAR ATAC EIRELI,34408066000137,3.522121e+43,0.0
1,35010024167891151040693B PIRUL POP MANIA MAXXI...,3501002416,7891151040693,17891151040690,B PIRUL POP MANIA MAXXI LUCCAS NETO,1.0,12.69,12.69,12.69,12.69,...,0.79,0,0,267167,JJGG-DISTRIBUIDORA ASSIS ALIMENTOS LTDA,9553327000165,CASA DA MAMAE LOJA DE DEP VAR ATAC EIRELI,34408066000137,3.522121e+43,0.0
2,35010024117891151040983B PIRUL POP MANIA MAXXI...,3501002411,7891151040983,7891151040983,B PIRUL POP MANIA MAXXI MELANC 24UN,2.0,12.69,12.69,12.69,25.38,...,1.58,0,0,267167,JJGG-DISTRIBUIDORA ASSIS ALIMENTOS LTDA,9553327000165,CASA DA MAMAE LOJA DE DEP VAR ATAC EIRELI,34408066000137,3.522121e+43,0.0


In [166]:
for i in sql:
    print(i[0])

i
r
e
e
d
a
r
c
u
t
t
c
n
c
i
a
i
a
i
i
a
i
i
a
p
a
c
a
f
n
f
c
l
c
k
f


In [167]:
sql = sql.values.tolist()
conn = psycopg2.connect(host='localhost', user='postgres',password= 'henrique',database = 'maindata',port= '5432')
cursor= conn.cursor()
for i in sql:
    cursor.execute('''INSERT INTO nfedatainformations( id,reference, ean, eantributed, description, amount,realcost, costcalculate, unitvalue, totalvalue,
                        tipe, cfop, ncm, cst, icmsbase, aliqicms, icmsvalue, aliqipi, ipivalue, ivamvavalue, aliqicmst,
                        icmstbase, icmstvalue, aliqpis, pisvalue, aliqcoffins, coffinsvalue, aliqfrete ,fretevalue, nfenum, fornecedor, cnpjforn, lojaentrada, cnpjlojaentrada, keynfe, frete_value) VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)''', 
                        (i[0],i[1], i[2],i[3], i[4], i[5], i[6], i[7], i[8], i[9], i[10], i[11], i[12], i[13], i[14], i[15], i[16], i[17], i[18], i[19], i[20], i[21], i[22], i[23], i[24], i[25], i[26], i[27],i[28],i[29],i[30],i[31], i[32] , i[33], i[34], i[35]))
    conn.commit()